In [2]:
# -*- coding: utf-8 -*-
"""
Stage 2 - Binary Classification (Grid Search, KNN from scratch) using PyTorch
Comparison models: 1R (decision stump), Decision Tree, Random Forest, SVM
Binary labels: quality >= 7 -> 1 else 0
Experiments: 80/20 and 70/30 stratified splits
Distance metrics: euclidean, manhattan, minkowski (p=3)
Grid Search over odd k in [1, 3, 5, ..., 49]
5-fold Stratified CV, tuning metric: average F1
Saves results to stage2_results_grid_allmodels.csv and confusion matrices:
 - cm_best_80_grid_allmodels.png
 - cm_best_70_grid_allmodels.png
"""

import numpy as np
import pandas as pd
import torch
import csv
import time
import random
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix
)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from ucimlrepo import fetch_ucirepo



# Device selection for PyTorch KNN
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
print("Stage 2 Classification (Grid Search) with model comparisons")
print("Starting...\n")

# ------------------------
# Distance functions
# ------------------------
def compute_distances(x, X_train, metric="euclidean", p=3):
    if metric == "euclidean":
        return torch.norm(X_train - x, dim=1)
    elif metric == "manhattan":
        return torch.sum(torch.abs(X_train - x), dim=1)
    elif metric == "minkowski":
        return torch.sum(torch.abs(X_train - x) ** p, dim=1) ** (1.0 / p)
    else:
        raise ValueError("Unknown metric: {}".format(metric))

# ------------------------
# KNN Classifier (PyTorch)
# ------------------------
class KNNClassifierTorch:
    def __init__(self, k=5, distance="euclidean", weights="uniform", p=3, print_freq=100):
        self.k = int(k)
        self.distance = distance
        self.weights = weights
        self.p = p
        self.print_freq = max(1, int(print_freq))

    def fit(self, X, y):
        # X: numpy array (N, D), y: array-like (N,)
        print("   Fit KNN: k={}, distance={}, weights={}".format(self.k, self.distance, self.weights))
        self.X_train = torch.tensor(X, dtype=torch.float32).to(device)
        y_arr = np.array(y)
        self.y_train = torch.tensor(y_arr.astype(np.int64), dtype=torch.float32).to(device)

    def predict_proba(self, X):
        X_t = torch.tensor(X, dtype=torch.float32).to(device)
        n = X_t.shape[0]
        probs = []
        for idx in range(n):
            if idx % self.print_freq == 0:
                print("      Predicting sample {}/{}".format(idx, n))
            x = X_t[idx]
            dists = compute_distances(x, self.X_train, metric=self.distance, p=self.p)
            knn_idx = torch.topk(dists, self.k, largest=False).indices
            knn_labels = self.y_train[knn_idx]
            if self.weights == "uniform":
                prob1 = torch.mean(knn_labels)
            else:
                eps = 1e-8
                weights = 1.0 / (dists[knn_idx] + eps)
                prob1 = (weights * knn_labels).sum() / weights.sum()
            probs.append(prob1.item())
        return np.array(probs)

    def predict(self, X, threshold=0.5):
        probs = self.predict_proba(X)
        return (probs >= threshold).astype(int)

# ------------------------
# Grid search over provided k grid
# ------------------------
def grid_search_knn(X, y, k_grid, distance, p=3, print_freq=100):
    """
    X: numpy array (N, D) - training data used for CV
    y: array-like (N,)
    k_grid: list of odd k values to evaluate
    Returns: dict {k: avg_f1}
    """
    results = {}
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    y_np = y if isinstance(y, np.ndarray) else y.values
    X_np = X

    print("Grid search for distance:", distance)
    for k in k_grid:
        print("  Testing k =", k)
        fold_scores = []
        fold_num = 1
        for train_idx, val_idx in skf.split(X_np, y_np):
            print("    Fold {}/5".format(fold_num))
            clf = KNNClassifierTorch(k=k, distance=distance, weights="uniform", p=p, print_freq=print_freq)
            clf.fit(X_np[train_idx], y_np[train_idx])
            y_val_pred = clf.predict(X_np[val_idx])
            f1 = f1_score(y_np[val_idx], y_val_pred, zero_division=0)
            fold_scores.append(f1)
            print("      Fold F1 =", f1)
            fold_num += 1
        avg_f1 = np.mean(fold_scores)
        results[k] = avg_f1
        print("  -> Avg F1 for k {} = {:.4f}\n".format(k, avg_f1))
    print("Grid search complete for distance:", distance)
    return results

# ------------------------
# Confusion matrix plot/save
# ------------------------
def save_confusion_matrix(y_true, y_pred, filename, title=None):
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion matrix (array):")
    print(cm)
    plt.figure(figsize=(5, 4))
    plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
    plt.title(title if title else "Confusion Matrix")
    plt.colorbar()
    ticks = np.arange(2)
    plt.xticks(ticks, ["0", "1"])
    plt.yticks(ticks, ["0", "1"])
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    thresh = cm.max() / 2.0 if cm.max() > 0 else 1
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()
    print("Saved confusion matrix to:", filename)

# ------------------------
# Evaluate library baseline models (1R via decision stump, decision tree, RF, SVM)
# ------------------------
def train_and_evaluate_baselines(X_train, y_train, X_test, y_test):
    results = {}

    # 1R as decision stump: DecisionTreeClassifier(max_depth=1)
    print(" Training 1R (decision stump)...")
    stump = DecisionTreeClassifier(max_depth=1, random_state=42)
    stump.fit(X_train, y_train)
    y_prob_stump = stump.predict_proba(X_test)[:, 1] if hasattr(stump, "predict_proba") else None
    y_pred_stump = stump.predict(X_test)
    results["1R"] = compute_metrics_and_store(y_test, y_pred_stump, y_prob_stump)

    # Decision Tree
    print(" Training Decision Tree...")
    dt = DecisionTreeClassifier(random_state=42)
    dt.fit(X_train, y_train)
    y_prob_dt = dt.predict_proba(X_test)[:, 1]
    y_pred_dt = dt.predict(X_test)
    results["DecisionTree"] = compute_metrics_and_store(y_test, y_pred_dt, y_prob_dt)

    # Random Forest
    print(" Training Random Forest...")
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    y_prob_rf = rf.predict_proba(X_test)[:, 1]
    y_pred_rf = rf.predict(X_test)
    results["RandomForest"] = compute_metrics_and_store(y_test, y_pred_rf, y_prob_rf)

    # SVM
    print(" Training SVM (probability=True)...")
    svm = SVC(probability=True, random_state=42)
    svm.fit(X_train, y_train)
    y_prob_svm = svm.predict_proba(X_test)[:, 1]
    y_pred_svm = svm.predict(X_test)
    results["SVM"] = compute_metrics_and_store(y_test, y_pred_svm, y_prob_svm)

    return results

def compute_metrics_and_store(y_true, y_pred, y_prob=None):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    if y_prob is None:
        rocauc = None
    else:
        try:
            rocauc = roc_auc_score(y_true, y_prob)
        except Exception:
            rocauc = None
    return {
        "accuracy": float(acc),
        "precision": float(prec),
        "recall": float(rec),
        "f1": float(f1),
        "roc_auc": float(rocauc) if rocauc is not None else None,
        "y_pred": np.array(y_pred),
        "y_test": np.array(y_true)
    }

# ------------------------
# Experiment runner using grid search plus baselines
# ------------------------
def run_classification_experiment_full_grid(X_scaled, y_binary, test_size, k_grid, print_freq=100):
    print("\nRunning experiment with test_size =", test_size)
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_binary, test_size=test_size, random_state=42, stratify=y_binary
    )
    print(" Data split done. Train size:", X_train.shape[0], "Test size:", X_test.shape[0])

    all_results = {}
    all_evals = {}

    # KNN models for each metric
    for metric in ["euclidean", "manhattan", "minkowski"]:
        print("\nKNN - Distance metric:", metric)
        cv_results = grid_search_knn(X_train, y_train, k_grid, distance=metric, p=3, print_freq=print_freq)
        best_k = sorted(cv_results.items(), key=lambda kv: (-kv[1], kv[0]))[0][0]
        print(" Best k selected for", metric, "=", best_k)

        knn = KNNClassifierTorch(k=best_k, distance=metric, weights="uniform", p=3, print_freq=print_freq)
        knn.fit(X_train, y_train)
        y_prob_knn = knn.predict_proba(X_test)
        y_pred_knn = (y_prob_knn >= 0.5).astype(int)

        acc = accuracy_score(y_test, y_pred_knn)
        prec = precision_score(y_test, y_pred_knn, zero_division=0)
        rec = recall_score(y_test, y_pred_knn, zero_division=0)
        f1 = f1_score(y_test, y_pred_knn, zero_division=0)
        try:
            rocauc = roc_auc_score(y_test, y_prob_knn)
        except Exception:
            rocauc = None

        all_results["KNN_{}".format(metric)] = {
            "best_k": best_k,
            "accuracy": float(acc),
            "precision": float(prec),
            "recall": float(rec),
            "f1": float(f1),
            "roc_auc": float(rocauc) if rocauc is not None else None
        }
        all_evals["KNN_{}".format(metric)] = {"y_test": np.array(y_test), "y_pred": np.array(y_pred_knn), "f1": f1}

        print(" Test results for KNN ({}) - k={}:".format(metric, best_k))
        print("   Accuracy: {:.4f}".format(acc))
        print("   Precision: {:.4f}".format(prec))
        print("   Recall: {:.4f}".format(rec))
        print("   F1-score: {:.4f}".format(f1))
        print("   ROC-AUC: {}".format(rocauc if rocauc is not None else "NA"))

    # Library baselines
    print("\nTraining and evaluating baseline library classifiers...")
    baseline_results = train_and_evaluate_baselines(X_train, y_train, X_test, y_test)
    for name, res in baseline_results.items():
        all_results[name] = {
            "accuracy": res["accuracy"],
            "precision": res["precision"],
            "recall": res["recall"],
            "f1": res["f1"],
            "roc_auc": res["roc_auc"]
        }
        all_evals[name] = {"y_test": res["y_test"], "y_pred": res["y_pred"], "f1": res["f1"]}

    # pick best model by test F1 among all
    best_model_name = None
    best_f1 = -1.0
    for name, evalinfo in all_evals.items():
        if evalinfo["f1"] > best_f1:
            best_f1 = evalinfo["f1"]
            best_model_name = name

    print("\nBest model on test set for this split is:", best_model_name, "with F1 =", best_f1)

    # save confusion matrix for the best model
    cm_filename = "cm_best_80_grid_allmodels.png" if test_size == 0.2 else "cm_best_70_grid_allmodels.png"
    best_eval = all_evals[best_model_name]
    save_confusion_matrix(best_eval["y_test"], best_eval["y_pred"], cm_filename,
                          title="Confusion Matrix - Best Model ({})".format(best_model_name))

    return all_results

# ------------------------
# Main
# ------------------------
print("Loading Wine Quality dataset...")
wine_quality = fetch_ucirepo(id=186)
X = wine_quality.data.features
y = wine_quality.data.targets

print("Converting to binary labels: quality >= 7 -> 1 else 0")
y_binary = (y >= 7).astype(int)

print("Scaling features...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Scaling completed.\n")

# grid of odd k values from 1 to 49
k_grid = list(range(1, 50, 2))  # [1,3,5,...,49]
print("k grid:", k_grid)

# grid search parameters
print_freq = 100
seed = 42

start_time = time.time()
# Run for 80/20
results_80 = run_classification_experiment_full_grid(X_scaled, y_binary, test_size=0.2, k_grid=k_grid, print_freq=print_freq)
# Run for 70/30
results_70 = run_classification_experiment_full_grid(X_scaled, y_binary, test_size=0.3, k_grid=k_grid, print_freq=print_freq)
end_time = time.time()
print("\nTotal runtime: {:.2f} seconds".format(end_time - start_time))

# ------------------------
# Save results to CSV
# ------------------------
out_csv = "stage2_results_grid_allmodels.csv"
print("Saving results to", out_csv)
with open(out_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Split", "Model", "Distance_or_Name", "Best_k_or_NA", "Accuracy", "Precision", "Recall", "F1", "ROC_AUC"])

    # 80/20
    for key, val in results_80.items():
        if key.startswith("KNN_"):
            metric = key.split("_", 1)[1]
            writer.writerow(["80/20", "KNN", metric, val.get("best_k"), val.get("accuracy"), val.get("precision"), val.get("recall"), val.get("f1"), val.get("roc_auc")])
        else:
            writer.writerow(["80/20", key, "NA", "NA", val.get("accuracy"), val.get("precision"), val.get("recall"), val.get("f1"), val.get("roc_auc")])

    # 70/30
    for key, val in results_70.items():
        if key.startswith("KNN_"):
            metric = key.split("_", 1)[1]
            writer.writerow(["70/30", "KNN", metric, val.get("best_k"), val.get("accuracy"), val.get("precision"), val.get("recall"), val.get("f1"), val.get("roc_auc")])
        else:
            writer.writerow(["70/30", key, "NA", "NA", val.get("accuracy"), val.get("precision"), val.get("recall"), val.get("f1"), val.get("roc_auc")])

print("Saved", out_csv)
print("Done.")


Device: cuda
Stage 2 Classification (Grid Search) with model comparisons
Starting...

Loading Wine Quality dataset...
Converting to binary labels: quality >= 7 -> 1 else 0
Scaling features...
Scaling completed.

k grid: [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49]

Running experiment with test_size = 0.2
 Data split done. Train size: 5197 Test size: 1300

KNN - Distance metric: euclidean
Grid search for distance: euclidean
  Testing k = 1
    Fold 1/5
   Fit KNN: k=1, distance=euclidean, weights=uniform
      Predicting sample 0/1040
      Predicting sample 100/1040
      Predicting sample 200/1040
      Predicting sample 300/1040
      Predicting sample 400/1040
      Predicting sample 500/1040
      Predicting sample 600/1040
      Predicting sample 700/1040
      Predicting sample 800/1040
      Predicting sample 900/1040
      Predicting sample 1000/1040
      Fold F1 = 0.6463700234192038
    Fold 2/5
   Fit KNN: k=1, distance=eucl

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/sklearn/base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


 Training SVM (probability=True)...


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Best model on test set for this split is: KNN_manhattan with F1 = 0.6509803921568628
Confusion matrix (array):
[[956  88]
 [ 90 166]]
Saved confusion matrix to: cm_best_80_grid_allmodels.png

Running experiment with test_size = 0.3
 Data split done. Train size: 4547 Test size: 1950

KNN - Distance metric: euclidean
Grid search for distance: euclidean
  Testing k = 1
    Fold 1/5
   Fit KNN: k=1, distance=euclidean, weights=uniform
      Predicting sample 0/910
      Predicting sample 100/910
      Predicting sample 200/910
      Predicting sample 300/910
      Predicting sample 400/910
      Predicting sample 500/910
      Predicting sample 600/910
      Predicting sample 700/910
      Predicting sample 800/910
      Predicting sample 900/910
      Fold F1 = 0.6312997347480106
    Fold 2/5
   Fit KNN: k=1, distance=euclidean, weights=uniform
      Predicting sample 0/910
      Predicting sample 100/910
      Predicting sample 200/910
      Predicting sample 300/910
      Predicting sa

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/sklearn/base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


 Training SVM (probability=True)...


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Best model on test set for this split is: RandomForest with F1 = 0.6481481481481481
Confusion matrix (array):
[[1512   55]
 [ 173  210]]
Saved confusion matrix to: cm_best_70_grid_allmodels.png

Total runtime: 107.69 seconds
Saving results to stage2_results_grid_allmodels.csv
Saved stage2_results_grid_allmodels.csv
Done.
